#Import potrebnih biblioteka

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Priprema podataka

In [ ]:
people = pd.DataFrame()
people['name'] = ['Alma_Cemer', 'Angelina_Jolie', 'Brad_Pitt', 'Kate_Winslet', 'Leonardo_DiCaprio', 'Natalie_Portman', 'Nedim_Omeragic',  'Scarlett_Johansson',  'Tom_Cruise', 'Will_Smith']
people['images'] = [113, 93, 95, 96, 97, 97, 113, 114, 93, 96]

image_paths = people.loc[people.index.repeat(people['images'])]
image_paths['image_path'] = 1 + image_paths.groupby('name').cumcount()
image_paths['image_path'] = image_paths.image_path.apply(lambda x: '{0:0>4}'.format(x))
image_paths['image_path'] = image_paths.name + "/" + image_paths.name + "_" + image_paths.image_path + ".jpg"
image_paths = image_paths.drop("images",1)


<ipython-input-13-2f21ace0f07d>:9: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  image_paths = image_paths.drop("images",1)


#Formiranje train i val setova


In [ ]:
names_list = image_paths['name'].value_counts().index.tolist()

data_frames_train = []
data_frames_val = []

for i in names_list:
  class_samples = image_paths[image_paths['name'] == i]
  train_set, val_set = train_test_split(class_samples, test_size=0.2, random_state=42)
  data_frames_train.append(train_set)
  data_frames_val.append(val_set)

multi_train = pd.concat(data_frames_train)
multi_train = multi_train.sample(frac = 1)

multi_val = pd.concat(data_frames_val)
multi_val = multi_val.sample(frac = 1)


#Funkcija za kreiranje direktorija

In [ ]:
import os
import shutil
from shutil import unpack_archive
def directory_mover(data,dir_name):
    co = 0
    for image in data.image_path:
        if not os.path.exists(os.path.join('/content/drive/MyDrive/',dir_name)):
            shutil.os.mkdir(os.path.join('/content/drive/MyDrive/',dir_name))

        data_type = data[data['image_path'] == image]['name']
        data_type = str(list(data_type)[0])
        if not os.path.exists(os.path.join('/content/drive/MyDrive/',dir_name,data_type)):
            shutil.os.mkdir(os.path.join('/content/drive/MyDrive/',dir_name,data_type))
        path_from = os.path.join('/content/drive/MyDrive/images_yolo/',image)
        path_to = os.path.join('/content/drive/MyDrive/',dir_name,data_type)
        shutil.copy(path_from, path_to)
        co += 1

    print('Moved {} images to {} folder.'.format(co,dir_name))

#Brisanje direktorija sa train i val podacima


In [ ]:
shutil.rmtree("/content/drive/MyDrive/face_recognition/train")
shutil.rmtree("/content/drive/MyDrive/face_recognition/val")


#Kreiranje direktorija sa train i val podacima

In [ ]:
#nije potrebno pokretati, oba foldera se nalaze na driveu
directory_mover(multi_train,"train_multi/")
directory_mover(multi_val,"val_multi/")


KeyboardInterrupt: 

#Naš ANN model


In [ ]:
from keras.utils import to_categorical
X_train_flat = []
for file in multi_train['image_path']:
    path = "/content/drive/MyDrive/images_yolo/" + str(file)
    im = Image.open(path)
    im_array = np.array(im)
    X_train_flat.append(im_array.flatten())

X_val_flat = []
for file in multi_val['image_path']:
    path = "/content/drive/MyDrive/images_yolo/" + str(file)
    im = Image.open(path)
    im_array = np.array(im)
    X_val_flat.append(im_array.flatten())

X_train_flat = np.array(X_train_flat)
X_val_flat = np.array(X_val_flat)


X_train_flat = X_train_flat.astype('float32') / 255
X_val_flat = X_val_flat.astype('float32') / 255
label_mapping = {'Alma_Cemer': 0, 'Angelina_Jolie': 1, 'Brad_Pitt': 2, 'Kate_Winslet': 3, 'Leonardo_DiCaprio': 4, 'Natalie_Portman': 5, 'Nedim_Omeragic': 6, 'Scarlett_Johansson': 7, 'Tom_Cruise': 8, 'Will_Smith': 9}
y_train_encoded = multi_train['name'].map(label_mapping)
y_val_encoded = multi_val['name'].map(label_mapping)

num_classes = len(label_mapping)
y_train_encoded = to_categorical(y_train_encoded, num_classes=num_classes)
y_val_encoded = to_categorical(y_val_encoded, num_classes=num_classes)

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
model = Sequential()
model.add(Dense(1024, input_shape=(X_train_flat.shape[1],), activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))

optimizer = tf.keras.optimizers.Adam()
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', mode='min', patience=7)

model_checkpoint = ModelCheckpoint('ANN_best_model.h5', monitor='val_loss', mode='min', save_best_only=True, verbose=1)

model.fit(X_train_flat, y_train_encoded, epochs=100, batch_size=16, validation_data = (X_val_flat, y_val_encoded), callbacks=[early_stopping, model_checkpoint])


Epoch 1/100
51/51 [==============================] - ETA: 0s - loss: 57.5810 - accuracy: 0.1211
Epoch 1: val_loss improved from inf to 16.45313, saving model to ANN_best_model.h5
51/51 [==============================] - 29s 553ms/step - loss: 57.5810 - accuracy: 0.1211 - val_loss: 16.4531 - val_accuracy: 0.1117
Epoch 2/100
50/51 [============================>.] - ETA: 0s - loss: 4.9954 - accuracy: 0.1950
Epoch 2: val_loss improved from 16.45313 to 3.02834, saving model to ANN_best_model.h5
51/51 [==============================] - 17s 333ms/step - loss: 4.9925 - accuracy: 0.1948 - val_loss: 3.0283 - val_accuracy: 0.1505
Epoch 3/100
51/51 [==============================] - ETA: 0s - loss: 2.2005 - accuracy: 0.2896
Epoch 3: val_loss did not improve from 3.02834
51/51 [==============================] - 2s 48ms/step - loss: 2.2005 - accuracy: 0.2896 - val_loss: 4.1110 - val_accuracy: 0.1699
Epoch 4/100
51/51 [==============================] - ETA: 0s - loss: 1.8566 - accuracy: 0.3945
Epoch 

#Naš CNN model

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint

multi_classifier = Sequential()

multi_classifier.add(Conv2D(32, (3, 3), input_shape = (224, 224, 3), activation = 'relu'))
multi_classifier.add(MaxPooling2D(pool_size = (2, 2)))
multi_classifier.add(Conv2D(64, (3, 3), activation = 'relu'))
multi_classifier.add(MaxPooling2D(pool_size = (2, 2)))
multi_classifier.add(Conv2D(128, (3, 3), activation = 'relu'))
multi_classifier.add(MaxPooling2D(pool_size = (2, 2)))
multi_classifier.add(Conv2D(256, (3, 3), activation = 'relu'))
multi_classifier.add(MaxPooling2D(pool_size = (2, 2)))
multi_classifier.add(Dropout(0.2))
multi_classifier.add(Flatten())

multi_classifier.add(Dense(units=1024, activation='relu'))
multi_classifier.add(Dropout(0.2))
multi_classifier.add(Dense(units=512, activation='relu'))
multi_classifier.add(Dropout(0.2))
multi_classifier.add(Dense(units=256, activation='relu'))

multi_classifier.add(Dense(units = 10, activation = 'softmax'))

optimizer=keras.optimizers.Adam(learning_rate = 0.00002)
multi_classifier.compile(optimizer = optimizer,loss = 'categorical_crossentropy', metrics = ['accuracy'])

train_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    shear_range=0.4,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)

multi_training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/face_recognition/train',
                                                 target_size = (224, 224),
                                                 batch_size = 16,
                                                 class_mode = 'categorical')
multi_val_set = test_datagen.flow_from_directory('/content/drive/MyDrive/face_recognition/val',
                                            target_size = (224, 224),
                                            batch_size = 16,
                                            class_mode = 'categorical')

early_stopping = EarlyStopping(monitor='val_loss', mode='min', patience=6)

model_checkpoint = ModelCheckpoint('CNN_best_model.h5', monitor='val_loss', mode='min', save_best_only=True, verbose=1)

multi_classifier.fit(x = multi_training_set, validation_data = multi_val_set, epochs = 100, callbacks=[early_stopping, model_checkpoint])

Found 800 images belonging to 10 classes.
Found 206 images belonging to 10 classes.
Epoch 1/100
50/50 [==============================] - ETA: 0s - loss: 2.3085 - accuracy: 0.0975
Epoch 1: val_loss improved from inf to 2.29511, saving model to CNN_best_model.h5
50/50 [==============================] - 19s 339ms/step - loss: 2.3085 - accuracy: 0.0975 - val_loss: 2.2951 - val_accuracy: 0.1117
Epoch 2/100
50/50 [==============================] - ETA: 0s - loss: 2.2974 - accuracy: 0.1063
Epoch 2: val_loss improved from 2.29511 to 2.28492, saving model to CNN_best_model.h5
50/50 [==============================] - 15s 299ms/step - loss: 2.2974 - accuracy: 0.1063 - val_loss: 2.2849 - val_accuracy: 0.1117
Epoch 3/100
50/50 [==============================] - ETA: 0s - loss: 2.2844 - accuracy: 0.1375
Epoch 3: val_loss improved from 2.28492 to 2.26720, saving model to CNN_best_model.h5
50/50 [==============================] - 15s 307ms/step - loss: 2.2844 - accuracy: 0.1375 - val_loss: 2.2672 - va

#VGG16 model

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint

num_classes = 10
input_size = (224, 224)

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(input_size[0], input_size[1], 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.2)(x)
predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    shear_range=0.4,
    zoom_range=0.2,
    horizontal_flip=True
)

train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/face_recognition/train',
    target_size=input_size,
    batch_size=16,
    class_mode='categorical'
)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

val_generator = val_datagen.flow_from_directory(
    '/content/drive/MyDrive/face_recognition/val',
    target_size=input_size,
    batch_size=16,
    class_mode='categorical'
)

early_stopping = EarlyStopping(monitor='val_loss', mode='min', patience=5)

model_checkpoint = ModelCheckpoint('VGG16_best_model.h5', monitor='val_loss', mode='min', save_best_only=True, verbose=1)


model.fit(train_generator, epochs=50, validation_data=val_generator, callbacks=[early_stopping, model_checkpoint])

58889256/58889256 [==============================] - 0s 0us/step
Found 800 images belonging to 10 classes.
Found 206 images belonging to 10 classes.
Epoch 1/50
50/50 [==============================] - ETA: 0s - loss: 2.2119 - accuracy: 0.3862
Epoch 1: val_loss improved from inf to 1.18450, saving model to VGG16_best_model.h5
50/50 [==============================] - 20s 286ms/step - loss: 2.2119 - accuracy: 0.3862 - val_loss: 1.1845 - val_accuracy: 0.6068
Epoch 2/50
50/50 [==============================] - ETA: 0s - loss: 0.9024 - accuracy: 0.6862
Epoch 2: val_loss improved from 1.18450 to 0.98965, saving model to VGG16_best_model.h5
50/50 [==============================] - 14s 282ms/step - loss: 0.9024 - accuracy: 0.6862 - val_loss: 0.9896 - val_accuracy: 0.6893
Epoch 3/50
50/50 [==============================] - ETA: 0s - loss: 0.6968 - accuracy: 0.7538
Epoch 3: val_loss did not improve from 0.98965
50/50 [==============================] - 14s 286ms/step - loss: 0.6968 - accuracy: 0.7

#VGGFace model

In [ ]:
!pip install git+https://github.com/yaledhlab/vggface.git
!pip install keras_applications

  Cloning https://github.com/yaledhlab/vggface.git to /tmp/pip-req-build-51i_cfx1
  Running command git clone --filter=blob:none --quiet https://github.com/yaledhlab/vggface.git /tmp/pip-req-build-51i_cfx1
  Resolved https://github.com/yaledhlab/vggface.git to commit b76539b7588bca69b0030ad7e4f985f877dc7c0a
  Preparing metadata (setup.py) ... done


In [ ]:
from collections import defaultdict
from keras_vggface.vggface import VGGFace
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ModelCheckpoint

num_classes = 10
input_size = (224, 224)

base_model = VGGFace(include_top=False, input_shape=(224, 224, 3), pooling='max')
x = base_model.output
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    shear_range=0.4,
    zoom_range=0.2,
    horizontal_flip=True
)

train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/face_recognition/train',
    target_size=input_size,
    batch_size=32,
    class_mode='categorical'
)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

val_generator = val_datagen.flow_from_directory(
    '/content/drive/MyDrive/face_recognition/val',
    target_size=input_size,
    batch_size=32,
    class_mode='categorical'
)

early_stopping = EarlyStopping(monitor='val_loss', mode='min', patience=5)

model_checkpoint = ModelCheckpoint('VGGFace_best_model.h5', monitor='val_loss', mode='min', save_best_only=True, verbose=1)


model.fit(train_generator, epochs=100, validation_data=val_generator, callbacks=[early_stopping, model_checkpoint])


Found 800 images belonging to 10 classes.
Found 206 images belonging to 10 classes.
Epoch 1/100
25/25 [==============================] - ETA: 0s - loss: 18.0141 - accuracy: 0.3812
Epoch 1: val_loss improved from inf to 3.60451, saving model to VGGFace_best_model.h5
25/25 [==============================] - 16s 570ms/step - loss: 18.0141 - accuracy: 0.3812 - val_loss: 3.6045 - val_accuracy: 0.8107


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 2/100
25/25 [==============================] - ETA: 0s - loss: 2.6647 - accuracy: 0.8338
Epoch 2: val_loss improved from 3.60451 to 0.95710, saving model to VGGFace_best_model.h5
25/25 [==============================] - 14s 568ms/step - loss: 2.6647 - accuracy: 0.8338 - val_loss: 0.9571 - val_accuracy: 0.9320
Epoch 3/100
25/25 [==============================] - ETA: 0s - loss: 1.1046 - accuracy: 0.9237
Epoch 3: val_loss did not improve from 0.95710
25/25 [==============================] - 14s 560ms/step - loss: 1.1046 - accuracy: 0.9237 - val_loss: 0.9964 - val_accuracy: 0.9369
Epoch 4/100
25/25 [==============================] - ETA: 0s - loss: 0.7576 - accuracy: 0.9413
Epoch 4: val_loss improved from 0.95710 to 0.63289, saving model to VGGFace_best_model.h5
25/25 [==============================] - 14s 555ms/step - loss: 0.7576 - accuracy: 0.9413 - val_loss: 0.6329 - val_accuracy: 0.9515
Epoch 5/100
25/25 [==============================] - ETA: 0s - loss: 0.4872 - accuracy: 0.96

#Yolo model za prepoznavanje lica

In [ ]:
%pip install ultralytics
import ultralytics

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import torch

from ultralytics import YOLO
yolo_results = []
output_root = '/content/drive/MyDrive/face_recognition'
model_yolo = YOLO('/content/drive/MyDrive/yolov8n-cls.pt')
results = model_yolo.train(data=output_root, epochs=50, patience=5, imgsz=224, batch=16)

Ultralytics YOLOv8.1.9 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=classify, mode=train, model=/content/drive/MyDrive/yolov8n-cls.pt, data=/content/drive/MyDrive/face_recognition, epochs=50, time=None, patience=5, batch=16, imgsz=224, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, 

100%|██████████| 6.23M/6.23M [00:00<00:00, 117MB/s]


AMP: checks passed ✅


train: Scanning /content/drive/MyDrive/face_recognition/train... 800 images, 0 corrupt: 100%|██████████| 800/800 [00:00<?, ?it/s]
val: Scanning /content/drive/MyDrive/face_recognition/val... 206 images, 0 corrupt: 100%|██████████| 206/206 [00:00<?, ?it/s]


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000714, momentum=0.9) with parameter groups 26 weight(decay=0.0), 27 weight(decay=0.0005), 27 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 224 train, 224 val
Using 2 dataloader workers
Logging results to runs/classify/train
Starting training for 50 epochs...

      Epoch    GPU_mem       loss  Instances       Size


...

      32/50     0.396G    0.05963         16        224: 100%|██████████| 50/50 [00:08<00:00,  5.89it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 7/7 [00:00<00:00,  8.89it/s]

                   all      0.961          1
Stopping training early as no improvement observed in last 5 epochs. Best results observed at epoch 27, best model saved as best.pt.
To update EarlyStopping(patience=5) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



32 epochs completed in 0.121 hours.
Optimizer stripped from runs/classify/train/weights/last.pt, 3.0MB
Optimizer stripped from runs/classify/train/weights/best.pt, 3.0MB

Validating runs/classify/train/weights/best.pt...
Ultralytics YOLOv8.1.9 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8n-cls summary (fused): 73 layers, 1447690 parameters, 0 gradients, 3.3 GFLOPs
train: /content/drive/MyDrive/face_recognition/train... found 800 images in 10 classes ✅ 
val: /content/drive/MyDrive/face_recognition/val... found 206 images in 10 classes ✅ 
test: None...


               classes   top1_acc   top5_acc: 100%|██████████| 7/7 [00:01<00:00,  6.07it/s]


                   all      0.981      0.995
Speed: 0.1ms preprocess, 1.7ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train
Results saved to runs/classify/train


In [ ]:
metrics = model_yolo.val()

Ultralytics YOLOv8.1.9 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8n-cls summary (fused): 73 layers, 1447690 parameters, 0 gradients, 3.3 GFLOPs
train: /content/drive/MyDrive/face_recognition/train... found 800 images in 10 classes ✅ 
val: /content/drive/MyDrive/face_recognition/val... found 206 images in 10 classes ✅ 
test: None...


val: Scanning /content/drive/MyDrive/face_recognition/val... 206 images, 0 corrupt: 100%|██████████| 206/206 [00:00<?, ?it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 13/13 [00:03<00:00,  3.92it/s]


                   all      0.981      0.995
Speed: 0.2ms preprocess, 6.5ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train2


##Provjera modela na slici

In [ ]:
results = model_yolo('Tom_Cruise_0008.jpg')


image 1/1 /content/Tom_Cruise_0008.jpg: 224x224 Tom_Cruise 1.00, Brad_Pitt 0.00, Nedim_Omeragic 0.00, Kate_Winslet 0.00, Natalie_Portman 0.00, 4.3ms
Speed: 3.2ms preprocess, 4.3ms inference, 0.0ms postprocess per image at shape (1, 3, 224, 224)


#Real-time photo recognition

In [ ]:
pip install opencv-python

In [ ]:
import numpy as np
import cv2
print("OpenCV-Python Version {}".format(cv2.__version__))

OpenCV-Python Version 4.8.0


In [ ]:
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.1.9 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.3/78.2 GB disk)


In [ ]:
from ultralytics import YOLO
import cv2
yolo_model = YOLO("/content/drive/MyDrive/yolov8n-face.pt")
model_yolo_cls = YOLO("/content/drive/MyDrive/yoloclsbest.pt")

WARNING ⚠️ /content/drive/MyDrive/yolov8n-face.pt appears to require 'omegaconf', which is not in ultralytics requirements.
AutoInstall will run now for 'omegaconf' but this feature will be removed in the future.
Recommend fixes are to train a new model using the latest 'ultralytics' package or to run a command with an official YOLOv8 model, i.e. 'yolo predict model=yolov8n.pt'
requirements: Ultralytics requirement ['omegaconf'] not found, attempting AutoUpdate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 14.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144554 sha256=d2bbbdf45804b8a778fc19813775a176783ef3a497edcd33e4ab114f651df30c
  Stored in directory: /tmp/pip-ephem-wheel-cache-wkef02ee/wheels/12/93/dd/1f6a127e

In [ ]:
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time
from google.colab.patches import cv2_imshow

In [ ]:
from IPython.display import Image
from tensorflow.keras.preprocessing import image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))

  results = yolo_model(filename, verbose=False)

  boxes = results[0].boxes
  img = cv2.imread(filename)

  for box in boxes:
    top_left_x = int(box.xyxy.tolist()[0][0])
    top_left_y = int(box.xyxy.tolist()[0][1])
    bottom_right_x = int(box.xyxy.tolist()[0][2])
    bottom_right_y = int(box.xyxy.tolist()[0][3])
    cropped_image = img[top_left_y:bottom_right_y, top_left_x:bottom_right_x]
    image2 = cv2.resize(cropped_image, (224, 224))
    cv2_imshow(image2)
    img_array = image.img_to_array(image2)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= .255

    #Ispod se koristi YOLO model za predikciju, ostali modeli su zakomentarisani
    #Yolo
    predictions2 = model_yolo_cls(image2, verbose=False)
    #NasaCNN
    #predictions2 = multi_classifier.predict(img_array)
    #VGGFace
    #predictions2 = model.predict(img_array, verbose=False)

    #Index za ostale modele
    #predicted_class_index2 = np.argmax(predictions2)

    #Index za yolo
    predicted_class_index2 = predictions2[0].probs.top1

    predicted_label = people['name'][predicted_class_index2]
    #Confidence za ostale modele
    #confidence = np.max(predictions2)

    #Confidence za yolo
    confidence = predictions2[0].probs.data[predicted_class_index2]

    if(confidence < 0.7):
      predicted_label = "Unknown person"

    print("Predicted Label:", predicted_label, confidence)

except Exception as err:
  print(str(err))

#Real-time video recognition

In [ ]:
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time

##Pomocne funkcije


In [ ]:
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  image_bytes = b64decode(js_reply.split(',')[1])
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  bbox_PIL.save(iobuf, format='png')
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes


def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;

    var pendingResolve = null;
    var shutdown = false;

    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }

    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }

    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);

      const modelOut = document.createElement('div');
      modelOut.innerHTML = "Status:";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);

      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);

      const instruction = document.createElement('div');
      instruction.innerHTML =
          '' +
          'When finished, click here or on the video to stop this demo';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };

      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640;
      captureCanvas.height = 480;
      window.requestAnimationFrame(onAnimationFrame);

      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();

      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }

      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }

      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;

      return {'create': preShow - preCreate,
              'show': preCapture - preShow,
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)

def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

##Real-time prepoznavanje

In [ ]:
from tensorflow.keras.preprocessing import image
video_stream()
label_html = 'Capturing...'
bbox = ''
count = 0
while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break

    frame = js_to_image(js_reply["img"])

    bbox_array = np.zeros([480,640,4], dtype=np.uint8)

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = yolo_model(rgb, verbose=False)
    boxes = results[0].boxes
    for box in boxes:
      top_left_x = int(box.xyxy.tolist()[0][0])
      top_left_y = int(box.xyxy.tolist()[0][1])
      bottom_right_x = int(box.xyxy.tolist()[0][2])
      bottom_right_y = int(box.xyxy.tolist()[0][3])
      cropped_image2 = rgb[top_left_y:bottom_right_y, top_left_x:bottom_right_x]
      image3 = cv2.resize(cropped_image2, (224, 224))
      img_array2 = image.img_to_array(image3)
      img_array2 = np.expand_dims(img_array2, axis=0)
      img_array2 /= .255

      #Yolo
      predictions2 = model_yolo_cls(image3, verbose=False)
      #NasaCNN
      #predictions2 = multi_classifier.predict(img_array)
      #VGGFace
      #predictions2 = model.predict(img_array2, verbose=False)
      #multi_classifier.predict(image2)
      #print(predictions2)

      #Index za ostale modele
      #predicted_class_index2 = np.argmax(predictions2)

      #Index za yolo
      predicted_class_index2 = predictions2[0].probs.top1

      predicted_label2 = people['name'][predicted_class_index2]
      #Confidence za ostale modele
      #confidence = np.max(predictions2)

      #Confidence za yolo
      confidence = predictions2[0].probs.data[predicted_class_index2]


      if(confidence > 0.7):
        bbox_array = cv2.rectangle(bbox_array, (top_left_x, top_left_y), (bottom_right_x, bottom_right_y), (50, 200, 129), 2)
        bbox_array = cv2.putText(bbox_array, "{} [{:.2f}]".format(predicted_label2, predictions2[0].probs.data[predicted_class_index2]),
                        (top_left_x, top_left_y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                        (50, 200, 129), 2)
      else:
        bbox_array = cv2.rectangle(bbox_array, (top_left_x, top_left_y), (bottom_right_x, bottom_right_y), (50, 200, 129), 2)
        bbox_array = cv2.putText(bbox_array, "{} [{:.2f}]".format('Unknown person', 1),
                        (top_left_x, top_left_y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                        (50, 200, 129), 2)

    bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
    bbox_bytes = bbox_to_bytes(bbox_array)
    bbox = bbox_bytes


<IPython.core.display.Javascript object>